<a href="https://colab.research.google.com/github/anandhc6/Assignment-2/blob/main/Assignment2_Part_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Downloading the dataset as zip folder

!gdown https://storage.googleapis.com/wandb_datasets/nature_12K.zip

Downloading...
From: https://storage.googleapis.com/wandb_datasets/nature_12K.zip
To: /content/nature_12K.zip
100% 3.82G/3.82G [00:18<00:00, 205MB/s]


In [ ]:
# unzipping the datset

!unzip -q nature_12K.zip

In [ ]:
#Installing wand

!pip install wandb --upgrade

     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 181 kB 45.3 MB/s 
     |████████████████████████████████| 144 kB 51.2 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=da2f19454dbbb93d33bd34ee2de61505f9e65c344f42e0131af26ac64a877ea2
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
#Importing necessary packages

import pathlib
import numpy as np
from PIL import Image
import tensorflow.keras as tfk
import os
import glob
import wandb
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Activation, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from wandb.keras import WandbCallback
import matplotlib.pyplot as plt
import math

In [ ]:
# Data preperation

def datagen(batch_size, augment_data):

  Data_dir=pathlib.Path('inaturalist_12K') 
  # augment_data=False
  train_path = os.path.join(Data_dir, "train")
  test_path = os.path.join(Data_dir, "val")

  if augment_data:
    train_rawdata = ImageDataGenerator(rescale=1./255,
                                      rotation_range=90,
                                      zoom_range=0.2,
                                      shear_range=0.2,
                                      validation_split=0.1,
                                      horizontal_flip=True)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  else:
    train_rawdata = ImageDataGenerator(rescale=1./255, validation_split=0.1)
    test_rawdata = ImageDataGenerator(rescale=1./255)

  train_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="training",
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  val_data = train_rawdata.flow_from_directory(
      train_path, 
      target_size=(256, 256), 
      batch_size=batch_size, 
      subset="validation",
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  test_data = test_rawdata.flow_from_directory(
      test_path, 
      target_size=(256, 256), 
      batch_size=batch_size,
      color_mode="rgb",
      class_mode='categorical',
      shuffle=True,
      seed=42
      )
  return  train_data, val_data, test_data

In [ ]:
# predefined models

def define_model(model_name, activation_function_dense, dense_layer, dropout,image_size, pre_layer_train=None):
  
    #input_image_shape=(256,256,3)
    ip_shape=(image_size, image_size, 3)
    input_tens = tfk.Input(shape = ip_shape)

    # adding pretrained model without the top dense layer
    if model_name == 'ResNet50':
      pretrained_model = tfk.applications.ResNet50(include_top = False, weights='imagenet',input_tensor = input_tens)
    elif model_name == 'InceptionV3':
      pretrained_model = tfk.applications.InceptionV3(include_top = False, weights='imagenet',input_tensor = input_tens)
    elif model_name == 'InceptionResNetV2':
      pretrained_model = tfk.applications.InceptionResNetV2(include_top = False, weights='imagenet',input_tensor = input_tens)
    else:
      pretrained_model = tfk.applications.Xception(include_top = False, weights='imagenet',input_tensor = input_tens)

    for layer in pretrained_model.layers:
      layer.trainable=False
    
    model = tfk.models.Sequential()
    # pretrained model
    model.add(pretrained_model)
    # converting the feature map into a column vector
    model.add(Flatten()) 
    # adding a dense layer
    model.add(Dense(dense_layer, activation=activation_function_dense))
    # dropout
    model.add(Dropout(dropout)) 
    # softmax layer
    model.add(Dense(10, activation="softmax"))

    return model

In [ ]:
#Function called by sweep

def train():
    activation_func_dense = "relu"
    image_size=256
    config_defaults = {
            "augment_data": True,
            "batch_size": 64,
            "dropout": 0,
            "dense_layer": 128,
            "layer_freeze": 0,
            "pre_model":"ResNet50"
        }

    # Initializing new wandb run and saving hyperparameters,inputs
    wandb.init(config=config_defaults)
    config = wandb.config

    # Values obtained from wandb config(Sweep)
    augment_data = config.augment_data
    batch_size = config.batch_size
    dropout = config.dropout
    dense_layer = config.dense_layer
    layer_freeze = config.layer_freeze
    pre_model=config.pre_model

    # Run name with hyperparameters
    run_name = "model_{}_aug_{}_bs_{}_drop_{}_dense_{}_freeze_{}".format(pre_model, augment_data, batch_size, dropout, dense_layer, layer_freeze )
    print(run_name)

    # Data preperation
    train_data, val_data, test_data =datagen(batch_size, augment_data)
    print("Data Recieved")
    model=define_model(pre_model, activation_func_dense, dense_layer, dropout,image_size)
    print(model.count_params())
    print("Model Training done")

    # #Freeze all layers
    # for layer in model.pretrained_model.layers:
    #     layer.trainable=False
    # #model.trainable=False

    model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    # Early Stopping 
    earlyStopping = EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='min')

    train_step_size = train_data.n//train_data.batch_size
    print(train_step_size)
    val_step_size = val_data.n//val_data.batch_size
    print(val_step_size)

    model_det = model.fit(train_data,
          steps_per_epoch = train_step_size,
          validation_data = val_data,
          validation_steps = val_step_size,
          epochs=10, 
          callbacks=[WandbCallback(data_type="image", generator=val_data), earlyStopping, best_val_check],
          verbose=2)
    
    #For fine tuning, unfreeze certain layers
    if layer_freeze:
      fine_tune=math.floor((layer_freeze/100.0)*len(model.layers))
      for layer in model.layers[-fine_tune:]:
        layer.trainable=True

    print("Fine tuning")
    model.compile(optimizer=Adam(learning_rate=0.0001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

    model_det_fine = model.fit(train_data,
          steps_per_epoch = train_step_size,
          validation_data = val_data,
          validation_steps = val_step_size,
          epochs=10, 
          callbacks=[WandbCallback(data_type="image", generator=val_data), earlyStopping, best_val_check],
          verbose=2)
    
    test_loss, test_acc = model.evaluate(test_data)
    print('Test accuracy :', test_acc)
    wandb.log({'test_accuracy': test_acc, 'test_loss': test_loss})
    wandb.run.name = run_name
    wandb.run.save()
    wandb.run.finish()
    return model_det_fine
